<a href="https://colab.research.google.com/github/GaetanAm/MASTER-ESILV/blob/main/Simulation_Methods/3_Black_Scholes_Call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3 Black-Scholes Call

In [39]:
import numpy as np
import math

In [37]:
# Function to simulate asset price S_T under Black-Scholes model
def simulate_black_scholes(S0, r, sigma, T, N):
    """
    Simulates the terminal asset price S_T using the Black-Scholes model.

    Args:
        S0 (float): Initial stock price.
        r (float): Risk-free interest rate (annualized).
        sigma (float): Volatility (annualized).
        T (float): Time to maturity (in years).
        N (int): Number of simulations.

    Returns:
        np.array: Simulated asset prices at time T.
    """
    # Generate N standard normal variables for Brownian motion
    W_T = np.sqrt(T) * np.random.randn(N)

    # Compute the asset price at maturity using the Black-Scholes formula
    S_T = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * W_T)

    return S_T

# Function to estimate the price of a European call option using Monte Carlo
def price_call_monte_carlo(S0, K, r, sigma, T, N):
    """
    Prices a European call option using Monte Carlo simulation.

    Args:
        S0 (float): Initial stock price.
        K (float): Strike price.
        r (float): Risk-free interest rate.
        sigma (float): Volatility of the stock.
        T (float): Time to maturity (in years).
        N (int): Number of Monte Carlo simulations.

    Returns:
        float: Estimated option price.
    """
    # Simulate asset prices at maturity
    S_T = simulate_black_scholes(S0, r, sigma, T, N)

    # Compute the option payoff at maturity
    payoffs = np.maximum(S_T - K, 0)

    # Compute the discounted expectation of the payoff
    option_price = np.exp(-r * T) * np.mean(payoffs)

    return option_price

In [41]:
# Parameters for the Monte Carlo simulation
S0 = 100  # Initial stock price
K = 100   # Strike price (ATM)
r = 0.01  # Risk-free interest rate (1% per year)
sigma = 0.2  # Volatility (20% per year)
T = 6 / 12  # Time to maturity in years (6 months)
N = 100000  # Number of Monte Carlo simulations

# Compute the option price
option_price = price_call_monte_carlo(S0, K, r, sigma, T, N)

# Print the result
print(f"Estimated European Call Option Price: {option_price:.4f}")

Estimated European Call Option Price: 5.8663


In [38]:
def price_call_monte_carlo_confidence(S0, K, r, sigma, T, N):
    """
    Prices a European call option using Monte Carlo simulation and computes a 99% confidence interval.

    Args:
        S0 (float): Initial stock price.
        K (float): Strike price.
        r (float): Risk-free interest rate.
        sigma (float): Volatility.
        T (float): Time to maturity.
        N (int): Number of Monte Carlo simulations.

    Returns:
        tuple: (Estimated option price, lower bound of confidence interval, upper bound of confidence interval)
    """
    # Simulate asset prices at maturity
    S_T = simulate_black_scholes(S0, r, sigma, T, N)

    # Compute the call option payoffs
    payoffs = np.maximum(S_T - K, 0)

    # Compute the option price
    option_price = np.exp(-r * T) * np.mean(payoffs)

    # Compute standard error
    std_error = np.std(payoffs) / np.sqrt(N)

    # Compute confidence interval (99% level, z-score = 2.576)
    z_alpha = 2.576
    lower_bound = option_price - z_alpha * std_error
    upper_bound = option_price + z_alpha * std_error

    return option_price, lower_bound, upper_bound


In [40]:
def black_scholes_call(S0, K, r, sigma, T):
    """
    Computes the price of a European call option using the Black-Scholes formula.

    Args:
        S0 (float): Initial stock price.
        K (float): Strike price.
        r (float): Risk-free interest rate.
        sigma (float): Volatility.
        T (float): Time to maturity.

    Returns:
        float: Exact price of the European call option.
    """
    # Compute d1 and d2
    d1 = (math.log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)

    # Compute option price using cumulative normal distribution
    N_d1 = 0.5 * (1 + math.erf(d1 / math.sqrt(2)))
    N_d2 = 0.5 * (1 + math.erf(d2 / math.sqrt(2)))

    call_price = S0 * N_d1 - K * math.exp(-r * T) * N_d2

    return call_price
